# CRAWLERS - Remote Job

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import re
import bs4
import sys
import requests

#### Play with the settings...

In [2]:
pd.set_option('display.max_colwidth', 150)

In [3]:
pd.set_option("display.max_rows", None)

## WORKING NOMADS

### Technique used: Selenium 

This was a challenging one. But I did realise that Selenium is a wonder.

In [7]:
def WORKING_NOMADS():
    #EDIT THE URL at will   
    
    url = python_latam_anywhere = "https://www.workingnomads.com/jobs?location=anywhere,latin-america&tag=python"
    #url = sales_anywhere = "https://www.workingnomads.com/jobs?location=anywhere&category=sales"
    
    # 1. Start the session
    driver = webdriver.Firefox()

    # 2. Take action on browser 
    driver.get(url)

    #4. Establish Waiting Strategy - 
    driver.implicitly_wait(0.5)

    #5 to use later... (to clean the text [blank soaces & n/])
    def TEXT_WASH(s):
        s = " ".join(s.split())
        s = re.sub(r'n/', '', s)
        return s

    #6. GETTING TITLES & JOBS -- GETTING RID OF EXPIRED POSITIONS...
    def GET_TITLES_ALL_JOBS():
        all_titles = []
        all_urls = []
        all_expired_urls = []
        #get all the jobs
        all_jobs = driver.find_elements(By.CSS_SELECTOR, ".job.clearfix.post .media-body h4 .open-button.ng-binding")
        #get the expired jobs
        expired_jobs = driver.find_elements(By.CSS_SELECTOR, "#jobs .job.expired .media-body h4 .open-button.ng-binding")
        for job in all_jobs: #add dirty titles & urls to their respective lists
            title = job.get_attribute("text")
            href = job.get_attribute("href")
            all_titles.append(title)
            all_urls.append(href)
        for job_expired in expired_jobs: #get expired links & add them to their list
            href_expired = job_expired.get_attribute("href")
            all_expired_urls.append(href_expired)
        #get rid of the expired links 
        curated_urls = [url for url in all_urls if url not in all_expired_urls] 
        #slice the list of titles to only get the ones that are not expired
        correct_but_dirty_titles = len(curated_urls)
        dirty_titles = all_titles[:correct_but_dirty_titles]
        #clean those dirty titles
        curated_titles = [TEXT_WASH(s) for s in dirty_titles]
        #Transform it into a df to improve readability
        df = pd.DataFrame({"JOB TITLES": curated_titles, "URLs": curated_urls})
        # Set the maximum column width to 1000 -> to avoid pd to truncate the URL
        #pd.set_option('display.max_colwidth', 1000)
        return df
    GET_TITLES_ALL_JOBS()

    driver.quit()

WORKING_NOMADS()

## WeWorkRemotely - Crawler

In [8]:
# CRITICAL!!! -> UNCOMMENT URL DEPENDING ON YOUR NEEDS...

#url = sales_url = 'https://weworkremotely.com/remote-jobs/search?term=&categories%5B%5D=9&region%5B%5D=0&region%5B%5D=4&job_listing_type%5B%5D=Contract&job_listing_type%5B%5D=Full-Time'
#url = software_eng_url = 'https://weworkremotely.com/remote-jobs/search?term=&categories%5B%5D=17&categories%5B%5D=18&categories%5B%5D=4&region%5B%5D=0&region%5B%5D=4&job_listing_type%5B%5D=Full-Time'
url = python_url = 'https://weworkremotely.com/remote-jobs/search?term=Python&button=&categories%5B%5D=17&categories%5B%5D=18&categories%5B%5D=4&region%5B%5D=0&region%5B%5D=4&job_listing_type%5B%5D=Full-Time'

print('Searching...')
def make_soup(url):
    try:
        res = requests.get(url)
        res.raise_for_status()
    except Exception as e:
        print(f'An error occurred: {e}')
        sys.exit(1)
    return res


soup = bs4.BeautifulSoup(make_soup(url).text, 'html.parser')

#Getting every URL

def wwr_url():
    all_url = soup.find_all(href=re.compile("/remote-jobs/|/listings/")) #We use | to include both href
    rubish = ['https://weworkremotely.com/remote-jobs/search', 'https://weworkremotely.com/remote-jobs/new', 'https://weworkremotely.comhttps://weworkremotely.com/remote-jobs/search']
    urls_list = []
    curated_url = []
    for link in all_url:
        href = link.get('href')
        urls_list.append("https://weworkremotely.com" + href)
        curated_url = [url for url in urls_list if url not in rubish]
    return curated_url

#TITLES + URL = {TITLE:URL}
    
titles = soup.select('.title')
def Get_Title_Url():
    All_titles = []
    for title in titles:
        title_class = title.get_text()
        All_titles.append(title_class)
    #TITLE_URL = dict(zip(All_titles, wwr_url()))
    df = pd.DataFrame({"JOB TITLES": All_titles, "URLs": wwr_url()})
    # Set the maximum column width to 1000 -> to avoid pd to truncate the URL
    #pd.set_option('display.max_colwidth', 1000)
    return df

Get_Title_Url()

#Download Dictionary in CSV or send it to Notion?


Searching...


,JOB TITLES,URLs
0,Solar Consultant,https://weworkremotely.com/remote-jobs/myfavoritesolar-com-solar-consultant
1,Director of Marketing,https://weworkremotely.com/remote-jobs/podia-director-of-marketing
2,Sales & Marketing Officer,https://weworkremotely.com/remote-jobs/64-robots-sales-marketing-officer
3,Chief Marketing Officer,https://weworkremotely.com/remote-jobs/themeisle-chief-marketing-officer
4,Product Marketing Manager (B2B SaaS),https://weworkremotely.com/remote-jobs/salesmsg-product-marketing-manager-b2b-saas
5,Customer Acquisition Manager,https://weworkremotely.com/remote-jobs/interaction-design-foundation-customer-acquisition-manager
6,Cold-Email Marketing Manager,https://weworkremotely.com/remote-jobs/hypergen-cold-email-marketing-manager
7,Director of Growth / Business Development Director,https://weworkremotely.com/remote-jobs/yoko-co-director-of-growth-business-development-director
8,Head of Growth / Marketing for VC Funded Music AI Startup,https://weworkremotely.com/remote-jobs/soundsnap-beatopia-head-of-growth-marketing-for-vc-funded-music-ai-startup
9,Conversion Rate Optimization (CRO) Consultant,https://weworkremotely.com/remote-jobs/conversion-rate-experts-cre-conversion-rate-optimization-cro-consultant-1


# REMOTEOK

### Technique used: rss feed

#### Ok, so for this one most of the work has been done. Just needs to add a way to save each file in a different excel spreadsheet. Hopefully, there is a way to determine the PATH whereby these documents will be saved

In [9]:
#! python3
# searchpypi.py  - Opens several search results.

import requests
import sys
import pandas as pd
import bs4 
import lxml 
# CRITICAL!!! -> THIS IS ONLY FOR XML!!! NOT HTTP PARSERS...

rss = rss_remote_ok = 'https://remoteok.com/rss'
print('Searching...')

#Make the request sirr
def request_rss(rss):
    try:
        res = requests.get(rss)
        res.raise_for_status()
    except Exception as e:
        print(f'An error occurred: {e}')
        sys.exit(1)
    return res

#MAKE THE SOOOOOOUP...
def make_soup():
    soup = bs4.BeautifulSoup(request_rss(rss).text, 'lxml-xml')
    return soup

# Get those tags
def REMOTE_OK():
    items_new = []
    for item in make_soup().find_all('item'):
        # extract the values of the 5 different tags
        title = item.title.get_text(strip=True)
        tags = item.tags.get_text(strip=True)
        location = item.location.get_text(strip=True)
        pubDate = item.pubDate.get_text(strip=True)
        link = item.link.get_text(strip=True)
        #add them to a list of dic
        items_new.append({'title': title, 'tags': tags, 'location': location, 'pubDate': pubDate, 'link': link})
        # df...
        df = pd.DataFrame(items_new, index=range(1, len(items_new)+1))
        # Set the maximum column width to 1000 -> to avoid pd to truncate the URL
        pd.set_option('display.max_colwidth', 1000)
        # add the values to the list
        #Worldwide_jobs_NOTSAVED = df[df['location'].str.contains('Worldwide', case=False)]
        #Worldwide_LATAM_jobs_NOTSAVED = df[(df['location'].str.contains('Worldwide', case=False)) | (df['location'].str.contains('LATAM', case=False))]
        #DF_SAVED = df.to_csv('REMOTE_OK.csv', index=False)
        #Worldwide_jobs = Worldwide_jobs_NOTSAVED.to_csv('REMOTE_OK_worldwide.csv', index=False)
        #Worldwide_LATAM = Worldwide_LATAM_jobs_NOTSAVED.to_csv('REMOTE_OK_Worldwide_LATAM.csv', index=False)
    return df

REMOTE_OK()

Searching...


,title,tags,location,pubDate,link
1,Quantitative Developer,"developer, defi, crypto",Worldwide,2023-02-25T01:10:06+00:00,https://remoteOK.com/jobs/199868
2,Data Engineer EU CZE,"design, web3, defi",Remote - CZE,2023-02-25T00:51:14+00:00,https://remoteOK.com/jobs/199879
3,Developer Advocate,"developer, design, web3",Worldwide,2023-02-25T00:51:11+00:00,https://remoteOK.com/jobs/199859
4,Software Engineer,"software, web3, front-end",Worldwide,2023-02-25T00:51:03+00:00,https://remoteOK.com/jobs/199880
5,Backend Engineer,"web3, react, graphql",Worldwide,2023-02-25T00:51:03+00:00,https://remoteOK.com/jobs/199867
6,Non tech,"crypto, ethereum",Worldwide,2023-02-25T00:51:03+00:00,https://remoteOK.com/jobs/199860
7,Data Engineer Intern,"python, software, financial",Worldwide,2023-02-24T23:11:07+00:00,https://remoteOK.com/jobs/199886
8,Chief Open Source Officer,"web3, support, developer",Worldwide,2023-02-24T23:11:06+00:00,https://remoteOK.com/jobs/199883
9,Information Systems Manager,"manager, saas, web3",Worldwide,2023-02-24T23:11:05+00:00,https://remoteOK.com/jobs/199885
10,Head of Marketing Director of Marketing,"director, web3, cryptocurrency",Worldwide,2023-02-24T23:11:05+00:00,https://remoteOK.com/jobs/199884
